In [93]:
import pandas as pd
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\pnii\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

Filtrage basé sur le contenu :

Utilise les caractéristiques des articles (ou des utilisateurs) pour trouver des recommandations similaires.
* Avantages : Peut fournir des recommandations explicites et peut surmonter les problèmes de démarrage à froid.
* Inconvénients : Peut manquer de sérendipité et de diversité dans les recommandations.

Chargement des données

In [103]:
df = pd.read_csv('D:\\Projo\\Projet-Data-IA\\Merged_all_tables\\merged_final.csv', sep=',')
df.sort_values(by='averageRating', ascending=False, inplace=True)
df.drop(columns=['poster_path', 'backdrop_path', 'nconst_director'], inplace=True)
df

,titleId,title,averageRating,numVotes,startYear,runtimeMinutes,genre1,genre2,overview,popularity,production_companies_name,Director_name,Actors_Actresses
1225,tt0111161,Les évadés,9.3,2878611.0,1994.0,142.0,Drama,Drama,Framed in the 1940s for the double murder of h...,84.139,Castle Rock Entertainment,Frank Darabont,"Mark Rolston, James Whitmore, Larry Brandenbur..."
233,tt0068646,Le Parrain,9.2,2004826.0,1972.0,175.0,Crime,Drama,"Spanning the years 1945 to 1955, a chronicle o...",113.216,"Alfran Productions, Paramount",Francis Ford Coppola,"Robert Duvall, Richard Conte, Al Pacino, James..."
1667,tt0167260,Le Seigneur des anneaux : Le Retour du roi,9.0,1972573.0,2003.0,201.0,Action,Adventure,Aragorn is revealed as the heir to the ancient...,65.571,"New Line Cinema, WingNut Films, The Saul Zaent...",Peter Jackson,"Ian McKellen, Sean Astin, Ali Astin, Elijah Wo..."
2475,tt0468569,The Dark Knight : Le Chevalier noir,9.0,2861067.0,2008.0,152.0,Action,Crime,Batman raises the stakes in his war on crime. ...,81.582,"DC Comics, Isobel Griffiths, Syncopy, Warner B...",Christopher Nolan,"Cillian Murphy, Morgan Freeman, Gary Oldman, M..."
1149,tt0108052,La liste de Schindler,9.0,1445610.0,1993.0,195.0,Biography,Drama,The true story of how businessman Oskar Schind...,57.910,Amblin Entertainment,Steven Spielberg,"Ralph Fiennes, Caroline Goodall, Embeth Davidt..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,tt0119675,Mimic,6.0,55427.0,1997.0,105.0,Horror,Sci-Fi,A disease carried by common cockroaches is kil...,14.997,"Miramax, Dimension Films",Guillermo del Toro,"Josh Brolin, Alix Koromzay, Jeremy Northam, F...."
1432,tt0119173,À armes égales,6.0,86820.0,1997.0,125.0,Action,Drama,In response to political pressure from Senator...,16.764,"Caravan Pictures, Hollywood Pictures, Roger Bi...",Ridley Scott,"Kevin Gage, Jason Beghe, Daniel von Bargen, Da..."
2820,tt1034331,La loi et l'ordre,6.0,92080.0,2008.0,101.0,Crime,Mystery,Two veteran New York City detectives work to i...,10.665,"InVenture Entertainment, Nu Image, Millennium ...",Jon Avnet,"50 Cent, Alan Rosenberg, Al Pacino, Brian Denn..."
4140,tt2910814,The Signal,6.0,69168.0,2014.0,97.0,Drama,Mystery,"On a road trip, Nic and two friends are drawn ...",22.840,"Automatik Entertainment, Focus Features, Low S...",William Eubank,"Brenton Thwaites, Beau Knapp, Timothy Holmes, ..."


1. Pre-processing :

* Encodage.

In [64]:
cat_cols = ["genre1", "genre2", "production_companies_name", "Director_name", "Actors_Actresses"]

encoder = OneHotEncoder()

2. Calcul des caractéristiques :

3. Calcul de la similarité :


4. Recommandation :

In [94]:
# Définir les colonnes catégorielles et numériques
cat_cols = ["title", "genre1", "genre2", "production_companies_name", "Director_name", "Actors_Actresses", "overview"]
num_cols = ["numVotes", "startYear", "runtimeMinutes", "popularity"]

# Créer une classe de transformation personnalisée pour extraire les sentiments de l'aperçu
class SentimentExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.analyzer = SentimentIntensityAnalyzer()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        sentiments = []
        for overview in X["overview"]:
            sentiment = self.analyzer.polarity_scores(overview)
            sentiments.append(list(sentiment.values()))
        return sentiments

# Créer un pipeline de transformation pour les caractéristiques
feature_pipeline = FeatureUnion([
    ("tfidf", TfidfVectorizer()),
    ("sentiment", SentimentExtractor())
])

# Appliquer le TfidfVectorizer sur la colonne "overview"
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(df["overview"])

# Appliquer le SentimentExtractor sur la colonne "overview"
sentiment_extractor = SentimentExtractor()
X_sentiment = sentiment_extractor.fit_transform(df)

# Concaténer les résultats
X_features = sp.hstack((X_tfidf, X_sentiment), format='csr')

# Standardiser les colonnes numériques
scaler = StandardScaler()
X_numerical = scaler.fit_transform(df[num_cols])

# Spécifier les poids pour chaque colonne numérique
num_feature_weights = {
    "numVotes": 2,
    "startYear": 0,
    "runtimeMinutes": 0,
    "popularity": 2
}

# Appliquer la pondération aux caractéristiques numériques en utilisant les poids spécifiés
X_numerical_weighted = X_numerical.copy()  # Copier X_numerical pour éviter de modifier les données d'origine
for col, weight in num_feature_weights.items():
    col_index = num_cols.index(col)
    X_numerical_weighted[:, col_index] *= weight


# Concaténer les caractéristiques encodées et standardisées
X = sp.hstack((X_features, X_numerical), format='csr')

# Calculer la similarité cosinus entre les films
cosine_sim = cosine_similarity(X, X)

# Fonction de recommandation de films similaires
def recommend_movies(movie_title, cosine_sim=cosine_sim, df=df):
    idx = df[df['title'] == movie_title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Exclure le film lui-même
    movie_indices = [i[0] for i in sim_scores]
    return df['title'].iloc[movie_indices]

# Exemple d'utilisation : obtenir des recommandations pour un film spécifique
movie_title = "Gladiator"
recommended_movies = recommend_movies(movie_title)
print("Recommandations pour", movie_title, ":")
print(recommended_movies)

Recommandations pour Gladiator :
3558     Une bague pour deux
5022          Flora & Ulysse
4342    Big Time Adolescence
4937            Dating Amber
4897     L'appel de la forêt
Name: title, dtype: object


In [105]:
# Télécharger les ressources nécessaires pour NLTK
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Initialiser le stemmer et le lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Fonction pour appliquer le stemming et la lemmatisation à un texte
def preprocess_text(text):
    # Tokenization
    words = word_tokenize(text)
    # Remove stopwords
    words = [word for word in words if word.lower() not in stopwords.words('english')]
    # Stemming
    stemmed_words = [stemmer.stem(word) for word in words]
    # Lemmatization
    lemmatized_words = [lemmatizer.lemmatize(word) for word in stemmed_words]
    return ' '.join(lemmatized_words)

# Appliquer la fonction de prétraitement à la colonne "overview"
df['processed_overview'] = df['overview'].apply(preprocess_text)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\pnii\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\pnii\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pnii\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [127]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
from nltk.sentiment import SentimentIntensityAnalyzer
import scipy.sparse as sp
import numpy as np

# Définir les colonnes catégorielles et numériques
cat_cols = ["title", "genre1", "genre2", "production_companies_name", "Director_name", "Actors_Actresses", "processed_overview"]
num_cols = ["numVotes", "startYear", "runtimeMinutes", "popularity"]

# Créer une classe de transformation personnalisée pour extraire les sentiments de l'aperçu
class SentimentExtractor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.analyzer = SentimentIntensityAnalyzer()

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        sentiments = []
        for overview in X["processed_overview"]:
            sentiment = self.analyzer.polarity_scores(overview)
            sentiments.append(list(sentiment.values()))
        return sentiments

# Créer un pipeline de transformation pour les caractéristiques
feature_pipeline = FeatureUnion([
    ("tfidf", TfidfVectorizer()),
    ("sentiment", SentimentExtractor())
])

# Appliquer le TfidfVectorizer sur la colonne "overview"
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(df["processed_overview"])

# Appliquer le SentimentExtractor sur la colonne "overview"
sentiment_extractor = SentimentExtractor()
X_sentiment = sentiment_extractor.fit_transform(df)

# Concaténer les résultats
X_features = sp.hstack((X_tfidf, X_sentiment), format='csr')

# Standardiser les colonnes numériques
scaler = StandardScaler()
X_numerical = scaler.fit_transform(df[num_cols])

# Spécifier les poids pour chaque colonne numérique
num_feature_weights = {
    "numVotes": 1,
    "startYear": 1,
    "runtimeMinutes": 1,
    "popularity": 2,
}

# Appliquer la pondération aux caractéristiques numériques en utilisant les poids spécifiés
X_numerical_weighted = X_numerical.copy()  
for col, weight in num_feature_weights.items():
    col_index = num_cols.index(col)
    X_numerical_weighted[:, col_index] *= weight

# Créer un vecteur de poids pour les caractéristiques catégorielles
cat_feature_weights = {
    "title": 1,
    "genre1": 3,
    "genre2": 1,
    "production_companies_name": 1,
    "Director_name": 1,
    "Actors_Actresses": 2,
    "processed_overview": 3
}

# Créer un encodeur one-hot
encoder = OneHotEncoder(handle_unknown='ignore')

# Adapter l'encodeur aux données d'entraînement et le transformer
X_cat_encoded = encoder.fit_transform(df[cat_cols])

# Appliquer la pondération aux caractéristiques catégorielles en utilisant les poids spécifiés
for col, weight in cat_feature_weights.items():
    col_index = cat_cols.index(col)
    X_cat_encoded[:, col_index] *= weight

# Concaténer les caractéristiques encodées et standardisées
X = sp.hstack((X_features, X_numerical_weighted, X_cat_encoded), format='csr')

# Calculer la similarité cosinus entre les films
cosine_sim = cosine_similarity(X, X)

# Fonction de recommandation de films similaires
def recommend_movies(movie_title, cosine_sim=cosine_sim, df=df):
    idx = df[df['title'] == movie_title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:6]  # Exclure le film lui-même
    movie_indices = [i[0] for i in sim_scores]
    return df['title'].iloc[movie_indices]

# Exemple d'utilisation : obtenir des recommandations pour un film spécifique
movie_title = "La loi et l'ordre"
recommended_movies = recommend_movies(movie_title)
print("Recommandations pour", movie_title, ":")
print(recommended_movies)


Recommandations pour La loi et l'ordre :
258             Vivre et laisser mourir
201          Les diamants sont éternels
64                           Goldfinger
142             On ne vit que deux fois
24     James Bond 007 contre docteur No
Name: title, dtype: object
